# Estimate a multivariate ARMA process

The objective of the Use Case is to estimate a multivariate ARMA model
from a stationary time series using the maximum likelihood estimator
and a centered normal white noise.
The data can be a unique time series or several time series collected
in a process sample.

We estimate $(\underline{\beta}, \sigma^2)$ thanks to the
*ARMALikelihoodFactory* object and its method *build*, acting on a
time series or on a sample of time series. It produces a result of
type *ARMA*.
Note that no evaluation of selection criteria such as *AIC* and *BIC*
is done.

The synthetic data is generated from the 2-d ARMA model:
$$X_{0,t} - 0.5 X_{0,t-1} - 0.1 X_{1,t-1} = E_{0,t} - 0.4 E_{0,t-1}$$
$$X_{1,t} - 0.4 X_{0,t-1} - 0.5 X_{1,t-1} - 0.25 X_{0,t-2} = E_{1,t} - 0.4 E_{1,t-1}$$

with E the white noise:
$$ E \sim \mathcal{N} ([0,0], [0.1,0.2])$$



In [1]:
from __future__ import print_function
import openturns as ot

In [2]:
# Create a 2-d ARMA process
p = 2
q = 1
dim = 2

# Tmin , Tmax and N points for TimeGrid
dt = 1.0
size = 400
timeGrid = ot.RegularGrid(0.0, dt, size)

# white noise
cov = ot.CovarianceMatrix([[0.1, 0.0], [0.0, 0.2]])
whiteNoise = ot.WhiteNoise(ot.Normal([0.0] * dim, cov), timeGrid)

# AR/MA coefficients
ar = ot.ARMACoefficients(p, dim)
ar[0] = ot.SquareMatrix([[-0.5, -0.1], [-0.4, -0.5]])
ar[1] = ot.SquareMatrix([[0.0, 0.0], [-0.25, 0.0]])

ma = ot.ARMACoefficients(q, dim)
ma[0] = ot.SquareMatrix([[-0.4, 0.0], [0.0, -0.4]])

# ARMA model creation
arma = ot.ARMA(ar, ma, whiteNoise)
arma

class= ARMA timeGrid=class=RegularGrid name=Unnamed start=0 step=1 n=400 coefficients AR=class=ARMACoefficients, shift=0, value=class=SquareMatrix dimension=2 implementation=class=MatrixImplementation name=Unnamed rows=2 columns=2 values=[-0.5,-0.4,-0.1,-0.5], shift=1, value=class=SquareMatrix dimension=2 implementation=class=MatrixImplementation name=Unnamed rows=2 columns=2 values=[0,-0.25,0,0] coefficients MA=class=ARMACoefficients, shift=0, value=class=SquareMatrix dimension=2 implementation=class=MatrixImplementation name=Unnamed rows=2 columns=2 values=[-0.4,0,0,-0.4] noiseDistribution= class=Normal name=Normal dimension=2 mean=class=Point name=Unnamed dimension=2 values=[0,0] sigma=class=Point name=Unnamed dimension=2 values=[0.316228,0.447214] correlationMatrix=class=CorrelationMatrix dimension=2 implementation=class=MatrixImplementation name=Unnamed rows=2 columns=2 values=[1,0,0,1] state= class= ARMAState x= class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=2 dimension=2 data=[[-0.458508,-0.542927],[-0.661498,0.142981]] epsilon= class=Sample name=Unnamed implementation=class=SampleImplementation name=Unnamed size=1 dimension=2 data=[[-0.541164,0.421868]]

In [3]:
# Create a realization
timeSeries = ot.TimeSeries(arma.getRealization())

In [4]:
# Estimate the process from the previous realization
factory = ot.ARMALikelihoodFactory(p, q, dim)
factory.setInitialConditions(ar, ma, cov)

arma_est = ot.ARMA(factory.build(timeSeries))
print('Estimated ARMA= ', arma_est)

Estimated ARMA=  ARMA(X_{0,t} - 0.746975 X_{0,t-1} - 0.095574 X_{1,t-1} + 0.0700775 X_{0,t-2} + 0.0116417 X_{1,t-2} = E_{0,t} - 0.661952 E_{0,t-1} - 0.0138457 E_{1,t-1}
X_{1,t} - 0.312501 X_{0,t-1} - 0.529355 X_{1,t-1} - 0.14329 X_{0,t-2} - 0.0454044 X_{1,t-2} = E_{1,t} + 0.124083 E_{0,t-1} - 0.427033 E_{1,t-1}, E_t ~ Normal(mu = [0,0], sigma = [0.312543,0.429097], R = [[ 1         0.0451618 ]
 [ 0.0451618 1         ]]))
